# Minimal Color Sum

Izvor grafova i njihovih najboljih bojenja: https://mat.gsia.cmu.edu/COLOR/instances.html

## Podaci

### Parsiranje velikih grafova u  DIMACS formatu

In [4]:
import codecs

def parse_dimacs_to_list(graph_file_name):
    
    graph_list = []
    dim = 0

    with codecs.open(graph_file_name, encoding='ascii') as graph_text:
    
        for line in graph_text:
            if line.startswith('e'):
                line = line[:-2]  
                line = line.split(' ')
                del line[0]    
                int_list = list(map(int,line))
                dim = max(dim, int(int_list[0]), int(int_list[1]))

        print('Dimenzija: ',dim)

        for i in range(0,dim+1):
            graph_list.append([])
        

    #popunjavanje liste
    with codecs.open(graph_file_name, encoding='ascii') as graph_text:        

        for line in graph_text:
            if line.startswith('e'):
                #print(line)
                line = line[:-2]  
                line = line.split(' ')
                del line[0]    
                int_list = list(map(int,line))
                #print(int_list)
                #print(int_list[0],int_list[1])
                graph_list[int_list[0]].append(int_list[1])
                graph_list[int_list[1]].append(int_list[0])
                #print(graph_list)


    return graph_list


In [2]:
#
graph0 = [
  [1,2],
  [0,3,4,6],
  [0,2,5],
  [1],
  [1,5],
  [2,4],
  [1]
]



In [3]:
#mali (4)
graph1 = parse_dimacs_to_list('graph1.txt')
graph1.pop(0) 
print(graph1)

ValueError: invalid literal for int() with base 10: ''

In [ ]:
#srednji (6)
graph2 = parse_dimacs_to_list('graph2.txt')
print(graph2)

In [ ]:
#srednje boja mnogo cvorova (20)
graph3 = parse_dimacs_to_list('graph3.txt')

In [ ]:
#mnogo boja mnogo cvorova  (42)
graph4 = parse_dimacs_to_list('graph4.txt')

In [ ]:
#mnogo covrova malo boja (8)
graph5 = parse_dimacs_to_list('graph5.txt')
f = open("primer.txt", "a")
f.write(graph5)
f.close()
print(graph5)

In [5]:
#mnogo cvorova malo grana (9)
graph6 = parse_dimacs_to_list('graph6.txt')

ValueError: invalid literal for int() with base 10: ''

In [ ]:
bf_colors0 = list(range(1,7+2))
bf_colors1 = list(range(1,10+1))
bf_colors2 = list(range(1,47+2))
bf_colors3 = list(range(1,128+2))
bf_colors4 = list(range(1,128+2))
bf_colors5 = list(range(1,95+2))
bf_colors6 = list(range(1,120+2))



## Algoritmi

### Brute Force

In [ ]:





minSolution = float('inf')
def isFeasible(graph, solution):
    for i in range(len(graph)):
        for neigh in graph[i]:
            if(solution[i]==solution[neigh]):
                return False
            
    return True


def allPossibleSolutionsRecur (possibleColors, data, last, index,graph): 
    length = len(possibleColors) 
    global minSolution
    for i in range(length):
        data[index] = possibleColors[i] 
        if index==last:
            # Ovde uopste ne ulazi, nzm zasto
            if(isFeasible(graph,data)):
                if(sum(data)< minSolution):
                    minSolution = sum(data)
                
                
        else: 
            allPossibleSolutionsRecur(possibleColors, data, last, index+1,graph) 
            

    
def allPossibleSolutions(possibleColors,graph): 
    
    length = len(possibleColors) 
    data = [1] * (length)  
    allPossibleSolutionsRecur(possibleColors, data, length-1, 0,graph) 
    print(minSolution)
    
    
allPossibleSolutions(bf_colors1,graph1)
print('Najbolje',minSolution)

In [ ]:
print(bf_colors1)

### Pohlepni algoritam

In [ ]:



def bfs(graph, root):
  
    visited = [root]
    queue = [root]
  
    color_dict = [-1 for i in range(len(graph))]

    while queue:
    
        s = queue.pop(0)


        #ovo moze lepse vrv
        not_satisfied = True
        color = 1

      

        adj = list(map(lambda x : color_dict[x], graph[s]))
        #print ('adj:',adj, end = '\n')
        while not_satisfied:
            if color not in adj:
                color_dict[s] = color
                not_satisfied = False
            else: 
                color = color + 1
      
        for neighbour in graph[s]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)

    return color_dict


#color_dict = bfs(graph3,1)
#print(color_dict)

### Genetski algoritam

In [ ]:
import random

class Individual:
    
    
    def __init__(self,graph):
        self.graph = graph
        self.colors = self.initColors(graph)
        self.fitness = 0
        
    def __lt__(self,other):
        return self.fitness < other.fitness
    
    def findAjdColors(self,i):
        ajdColors = []
        for ajd in self.graph[i]:
            ajdColors.append(self.colors[ajd])
        return ajdColors
        
        
    def correctNonFeasible(self,all_possible):
        
        for i in range(len(self.graph)):
            ajdColors = self.findAjdColors(i)
            if(self.colors[i] in ajdColors):
                
                possibleColors = list(set(all_possible)-set(ajdColors))
                self.colors[i] = random.choice(possibleColors)
        
            
    def fitnessFunction(self,colors):
        value = 0
        for color in colors:
            value+=color
        return value
    
    def initColors(self,graph):
        colors = []
        numberOfColors = len(graph)
        for i in range(numberOfColors):
            colors.append(random.randrange(1,numberOfColors+1))
        return colors

    def __str__(self):
        print('Bojenje' + str(self.colors))
        

        
# Selekcija -> biramo jedinke iz populacije za generisanje naredne
# Koristimo turnirsku: za pocetak 6 turnira 
def tournamentSelection(population):
    min = float('inf')
    k=-1
    for j in range(6):
        j = random.randrange(100) # 100 -> broj populacije
        if population[j].fitness < min:
            min = population[j].fitness
            k=j
    
    return k



def rouletteSelection(population):
    totalFitness = sum([individual.fitness for individual in population])
    randomValue = random.randrange(0,totalFitness)
    currentSum = 0
    
    for i in range(len(population)):
        currentSum += population[i].fitness
        if currentSum > randomValue:
            return i
        
        
        
# Jednopoziciono ukrstanje
def crossover(parent1,parent2,child1,child2,all_possible):
    numberColors = len(parent1.colors)
    # biramo poziciju ukrstanja
    position = random.randrange(numberColors)
    # razmenjujemo
    # prvi deo bojenja
    for i in range(position):
        child1.colors[i]=parent1.colors[i]
        child2.colors[i]=parent2.colors[i]
    
    # drugi deo bojenja
    for j in range(position,numberColors):
        child1.colors[j]=parent2.colors[j]
        child2.colors[j]=parent1.colors[j]
        
    # menjamo da budu dopustiva
    child1.correctNonFeasible(all_possible)
    child2.correctNonFeasible(all_possible)
    
    
    
    
# Mutacija
# Verovatnoca za mutaciju 0.05 (parametar, probati druge)
def mutation(individual, all_possible):
    numberColors = len(individual.colors)
    for i in range(numberColors):
        if random.random() > 0.05:
            continue
        individual.colors[i] = random.choice(individual.colors)
        individual.correctNonFeasible(all_possible)
        
        
# Inicijalna populacija
# 100 jedinki u populaciji (parametar, probati druge)
# za
def initPopulation(graph,all_possible):
    population = []
    newPopulation = []
    for i in range(100):
        individual = Individual(graph)
        individual.initColors(graph)
        individual.correctNonFeasible(all_possible)
        individual.fitness = individual.fitnessFunction(individual.colors)
        population.append(individual)
        newPopulation.append(individual)
        
    return (population,newPopulation)



In [ ]:
# Sprovodimo Genetski algoritam
# 500 iteracija (probati druge parametre)




# Dobijamo inicijalnu populaciju
def Gen(graph, all_possible):
    population,newPopulation = initPopulation(graph,all_possible)



    for iteration in range(500):
        population.sort() # da bi nam prvi bio sa najboljim fitnesom tj minimalnom sumom
        # Elitizam, cuvamo 30 najboljih
        for i in range(30):
            newPopulation[i] = population[i]
        # Ostale zamenjujemo decom
        for i in range(30,100,2):
            ind1 = tournamentSelection(population)
            ind2 = tournamentSelection(population)
            crossover(population[ind1],population[ind2],newPopulation[i],newPopulation[i+1],all_possible)
            mutation(newPopulation[i],all_possible)
            mutation(newPopulation[i+1],all_possible)
            newPopulation[i].fitness = newPopulation[i].fitnessFunction(newPopulation[i].colors)
            newPopulation[i+1].fitness = newPopulation[i+1].fitnessFunction(newPopulation[i+1].colors)

        population=newPopulation

    population.sort()

    print('Resenje',population[0].fitness)
# Sa ruletskom se dobija 12




### Simulirano kaljenje

In [ ]:
from numpy import log as ln
import random

def isFeasible(graph, solution):
    for i in range(len(graph)):
        for neigh in graph[i]:
            if(solution[i]==solution[neigh]):
                return False
            
    return True

def findAjdColors(i,graph,colors):
        ajdColors = []
        for ajd in graph[i]:
            ajdColors.append(colors[ajd])
        return ajdColors
        
        
def correctNonFeasible(graph,colors):
        
    for i in range(len(graph)):
        ajdColors = findAjdColors(i,graph,colors)
        if(colors[i] in ajdColors):  
            possibleColors = list(set([1,2,3,4,5,6,7,8])-set(ajdColors))
            colors[i] = random.choice(possibleColors)
    


def initializeSolution(graph):
    colors = []
    numberOfColors = len(graph)
    for i in range(numberOfColors):
            colors.append(random.randrange(1,numberOfColors+1))
    
    if not isFeasible(graph,colors):
        correctNonFeasible(graph,colors)
        
    return colors

# racuna vrednost funkcije cilja
def solutionValue(solution):
    return sum(solution)
# biramo resenje iz okoline
# random odaberemo indeks i smanjimo mu vrednost boje
oldValue = 0
def adjacencySolution(solution,graph):
    global oldValue
    position = random.randrange(0,len(graph))
    oldValue = solution[position]
    oldPosition = position
    if solution[position]==1:
        solution[position]=solution[position]+1
    else:
        solution[position]=solution[position]-1
      
    #newColor = random.randrange(1,len(graph)+1)
    #solution[position] = newColor ---> nije susedno vrv
    if isFeasible(graph,solution):
        return position
    solution[position] = oldValue
    return -1
            
    
# vracamo staru vrednost ako ne dobijamo bolje resenje

def restore(position,solution):
    solution[position] = oldValue

In [ ]:
# Algoritam simuliranog kaljenja
def simulatedAnnealing(graph,solution,maxIters):
    currValue = solutionValue(solution)
    bestValue = currValue # na pocetku od pocetnog resenja 
    i=1
    # ponavljamo postupak dok ne dostignemo maksimalno iteracija
    while i<maxIters:
        # uzimamo resenje u okolini
        j = adjacencySolution(solution,graph)
        # ako nije dopustivo idemo na narednu iteraciju 
        if j<0: 
            continue
        newValue = solutionValue(solution)
        if newValue < currValue:
            currValue = newValue
        else:
            # Racunamo vrednost opadajuce fje
            # Uzimamo druge fje p
            
            p = 1.0 / i ** 0.5
           #p = ln(2) / ln(1+i) 
            q = random.uniform(0, 1) #treba nam random vr [0,1]
            if p > q:
                currValue = newValue
            else:
                restore(j, solution)
        if newValue < bestValue:
            bestValue = newValue
        i += 1
    return bestValue

In [ ]:
graph_list2 = [[3],[3],[3],[0,1,2,4],[3,5,6,7],[4],[4],[4]]
solution = initializeSolution(graph_list2)
value = simulatedAnnealing(graph_list2,solution,1000)
print('Najbolje resenje',value)

#### Hibridizacija

In [ ]:
import random

class Individual_hybrid:
    
    
    def __init__(self,graph):
        self.graph = graph
        self.colors = self.initColors(graph)
        self.fitness = 0
        self.oldValue = 0
        
    def __lt__(self,other):
        return self.fitness < other.fitness
    
    def findAjdColors(self,i):
        ajdColors = []
        for ajd in self.graph[i]:
            ajdColors.append(self.colors[ajd])
        return ajdColors
   

    def isFeasible(self):
        for i in range(len(self.graph)):
            for neigh in self.graph[i]:
                if(self.colors[i]==self.colors[neigh]):
                    return False
            
        return True
    
    # biramo resenje iz okoline
# random odaberemo indeks i smanjimo mu vrednost boje
   
    def adjacencySolution(self):
        position = random.randrange(0,len(self.graph))
        self.oldValue = self.colors[position]
        if self.colors[position]==1:
            self.colors[position]=self.colors[position]+1
        else:
            self.colors[position]=self.colors[position]-1
      
    #newColor = random.randrange(1,len(graph)+1)
    #solution[position] = newColor ---> nije susedno vrv
        if self.isFeasible():
            return position
        self.colors[position] = self.oldValue
        return -1

        
    def correctNonFeasible(self):
        
        for i in range(len(self.graph)):
            ajdColors = self.findAjdColors(i)
            if(self.colors[i] in ajdColors):
                
                possibleColors = list(set([1,2,3,4,5,6,7,8])-set(ajdColors))
                self.colors[i] = random.choice(possibleColors)
        
    
    def restore(self,position):
        self.colors[position] = self.oldValue

    def fitnessFunction(self,colors):
        value = 0
        for color in colors:
            value+=color
        return value
    
    def initColors(self,graph):
        colors = []
        numberOfColors = len(graph)
        for i in range(numberOfColors):
            colors.append(random.randrange(1,numberOfColors+1))
        return colors

    def __str__(self):
        print('Bojenje' + str(self.colors))
        


In [ ]:
def simulatedAnnealing(individual, maxIters):
    for i in range(maxIters):
        j = individual.adjacencySolution()
        if j < 0:
            continue
        newFitness = individual.fitnessFunction(individual.colors)
        if newFitness < individual.fitness:
            individual.fitness = newFitness
        else:
            p = 1.0 / (i + 1) ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                individual.fitness = newFitness
            else:
                individual.restore(j)

In [ ]:
# Sprovodimo Genetski algoritam
# 500 iteracija (probati druge parametre)

graph_list = [
  [1,2],
  [0,3,4,6],
  [0,2,5],
  [1],
  [1,5],
  [2,4],
  [1]
]

#graph_list2 = [[3],[3],[3],[0,1,2,4],[3,5,6,7],[4],[4],[4]]

# Dobijamo inicijalnu populaciju
population,newPopulation = initPopulation(graph_list2)



for iteration in range(500):
    population.sort() # da bi nam prvi bio sa najboljim fitnesom tj minimalnom sumom
    # Elitizam, cuvamo 30 najboljih
    for i in range(30):
        newPopulation[i] = population[i]
    # Ostale zamenjujemo decom
    for i in range(30,100,2):
        ind1 = tournamentSelection(population)
        ind2 = tournamentSelection(population)
        crossover(population[ind1],population[ind2],newPopulation[i],newPopulation[i+1])
        mutation(newPopulation[i])
        mutation(newPopulation[i+1])
        newPopulation[i].fitness = newPopulation[i].fitnessFunction(newPopulation[i].colors)
        newPopulation[i+1].fitness = newPopulation[i+1].fitnessFunction(newPopulation[i+1].colors)
    # sprovodimo kaljenje na najbolji u populaciji
    simulatedAnnealing(newPopulation[0], 100)
    population=newPopulation

population.sort()

print('Resenje',population[0].fitness)
# Sa ruletskom se dobija 12

### PSO

In [ ]:
import random
def isFeasiblePSO(graph, solution):
    for i in range(len(graph)):
        for neigh in graph[i]:
            if(solution[i]==solution[neigh]):
                return False
            
    return True

def findAjdColorsPSO(i,graph,colors):
        ajdColors = []
        for ajd in graph[i]:
            ajdColors.append(colors[ajd])
        return ajdColors
        
        
def correctNonFeasiblePSO(graph,colors,all_possible):
        
    for i in range(len(graph)):
        ajdColors = findAjdColorsPSO(i,graph,colors)
        if(colors[i] in ajdColors):  
            possibleColors = list(set(all_possible)-set(ajdColors))

            colors[i] = random.choice(possibleColors)
            
                


def initializeSolutionPSO(graph,all_possible):
    colors = []
    numberOfColors = len(graph)
    for i in range(numberOfColors):
            colors.append(random.randrange(1,numberOfColors+1))
    
    if not isFeasiblePSO(graph,colors):
        correctNonFeasiblePSO(graph,colors,all_possible)
        
    return colors

# racuna vrednost funkcije cilja
def solutionValuePSO(solution):
    return sum(solution)

In [ ]:
def init(numberVertex,position,velocity,bestValue,numberOfParticles,graph,all_possible):
    # za svaku cesticu pravimo bojenje
    globalBestValue = float("inf")
    for i in range(numberOfParticles):
        coloring = initializeSolutionPSO(graph,all_possible )
        position.append(coloring)
        bestValue.append(solutionValuePSO(coloring))
        velocity.append(0.0)
        if(solutionValuePSO(coloring)<globalBestValue):
            globalBestValue = solutionValuePSO(coloring)
            globalBest = coloring

    return globalBestValue,globalBest 

In [ ]:
#apdejtujemo vektor brzina
max_velocity = 10
def updateVelocity(position,velocity,bestValue,globalBestValue):
    # za svaku jedinku
    for i in range(len(velocity)):
        # po formuli 
        #              koliko pridajemo znajacnosti      pravac
        velocity[i] += 2.0*random.random()*(bestValue[i]-solutionValuePSO(position[i])) \
                    + 2.0*random.random()*(globalBestValue-solutionValuePSO(position[i]))
        if velocity[i] > max_velocity:
            velocity[i] = max_velocity
        if velocity[i] < -max_velocity:
            velocity[i] = - max_velocity

In [ ]:

def updatePosition(position,velocity,bestValue,globalBest,globalBestValue,graph,all_possible):
    # za svaku cesticu
    for i in range(len(position)):
        # prolazimo kroz svaki deo koloringa, na svaki broj dodajemo brzinu
        # isto formula
        for j in range(len(position[i])):
            position[i][j]+=int(velocity[i])#jer nam trebaju celi brojevi, necemo velocity jer bismo stalno zaokruzili i usporili
            if position[i][j] > len(graph):
                position[i][j] = len(graph)
            if position[i][j] < 1:
                position[i][j] = 1

        if not isFeasiblePSO(graph,position[i]):
            correctNonFeasiblePSO(graph,position[i],all_possible)
        value = solutionValuePSO(position[i])
        if value < bestValue[i]:
            bestValue[i]=value
        if value<globalBestValue:
            globalBestValue = value
            globalBest = position[i].copy()
        
        
    return globalBestValue  

def PSO(graph,numberVertex,numberOfParticles,maxIteration,all_possible):
    position = []   #niz svih bojenja
    velocity = []  # vektor brzina
    bestValue = [] #vektor najboljih pozicija za svaku cesticu
    
    globalBestValue,globalBest = init(len(graph),position,velocity,bestValue,numberOfParticles,graph,all_possible)
    for iteration in range(maxIteration):
        #apdejtujemo brzinu
        updateVelocity(position,velocity,bestValue,globalBestValue)
        globalBestValue = updatePosition(position,velocity,bestValue,globalBest,globalBestValue,graph,all_possible)
        
    print('Najbolje resenje',globalBestValue)

In [ ]:
PSO(graph5,len(graph5),5,550)

In [ ]:
info_map{
    'brute_force':{},
    'greedy':{},
    'gen_alg':{},
    'anneling':{},
    'pso':{}
    
}

In [ ]:
import time


In [ ]:
def timer(algorithm, a1, a2):
    start = time.perf_counter()

    algorithm(a1, a2)
    
    end = time.perf_counter()
    print(end - start)

In [ ]:
timer(fn, 2, 2)

In [ ]:
import time 

### Rezultati

In [ ]:
#possible color vectors prep
#doduse ad-hoc za sad
bf_colors0 = list(range(1,7+2))
bf_colors1 = list(range(1,10+2))
bf_colors2 = list(range(1,47+2))
bf_colors3 = list(range(1,128+2))
bf_colors4 = list(range(1,128+2))
bf_colors5 = list(range(1,95+2))
bf_colors6 = list(range(1,120+2))

color_list = [bf_colors0, bf_colors1, bf_colors2, bf_colors3, bf_colors4, bf_colors5, bf_colors6]
graphs = [graph0, graph1, graph2, graph3, graph4, graph5, graph6]

zipped = zip(color_list, graphs)

for z in zipped:
    start = time.perf_counter()
    allPossibleSolutions(z[0],z[1])
    end = time.perf_counter()
    print(end - start)


In [ ]:
for g in [graph0, graph1, graph2, graph3, graph4, graph5, graph6]:
    start = time.perf_counter()

    bfs(g, 1)

    end = time.perf_counter()
    print(end - start)

In [ ]:
bf_colors0 = list(range(1,7+2))
bf_colors1 = list(range(1,10+2))
bf_colors2 = list(range(1,47+2))
bf_colors3 = list(range(1,128+2))
bf_colors4 = list(range(1,128+2))
bf_colors5 = list(range(1,95+2))
bf_colors6 = list(range(1,120+2))

color_list = [bf_colors0, bf_colors1, bf_colors2, bf_colors3, bf_colors4, bf_colors5, bf_colors6]
graphs = [graph0, graph1, graph2, graph3, graph4, graph5, graph6]

zipped = zip(color_list, graphs)

for z in zipped:
    start = time.perf_counter()
    
    PSO(z[1],len(z[1]),5,550,z[0])
    end = time.perf_counter()
    print(end - start)






In [ ]:
bf_colors0 = list(range(1,7+2))
bf_colors1 = list(range(1,10+2))
bf_colors2 = list(range(1,47+2))
bf_colors3 = list(range(1,128+2))
bf_colors4 = list(range(1,128+2))
bf_colors5 = list(range(1,95+2))
bf_colors6 = list(range(1,120+2))

color_list = [bf_colors0, bf_colors1, bf_colors2, bf_colors3, bf_colors4, bf_colors5, bf_colors6]
graphs = [graph0, graph1, graph2, graph3, graph4, graph5, graph6]

zipped = zip(color_list, graphs)

for z in zipped:
    start = time.perf_counter()
    
    Gen(z[1],z[0])
    end = time.perf_counter()
    print(end - start)

    
    

In [ ]:
bf_colors0 = list(range(1,7+1))
bf_colors0

In [ ]:
print(list(range(1,10+1)))